In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv("./Datasets/Fake_Real_Data.csv")
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [3]:
df.shape

(9900, 2)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9900 entries, 0 to 9899
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    9900 non-null   object
 1   label   9900 non-null   object
dtypes: object(2)
memory usage: 154.8+ KB


In [5]:
df.describe()

,Text,label
count,9900,9900
unique,9865,2
top,Highlights: The Trump presidency on April 13 a...,Fake
freq,8,5000


In [6]:
df.label.value_counts()

label
Fake    5000
Real    4900
Name: count, dtype: int64

In [7]:
df.label = df.label.map({
    "Fake" : 0,
    "Real" : 1
})

In [8]:
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,0
1,U.S. conservative leader optimistic of common ...,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",1
3,Court Forces Ohio To Allow Millions Of Illega...,0
4,Democrats say Trump agrees to work on immigrat...,1


In [10]:
import spacy
import gensim.downloader as api
wv = api.load("word2vec-google-news-300")
nlp = spacy.load("en_core_web_lg")
def preprocessing(text):
    doc = nlp(text)
    filter = []
    for i in doc:
        if not i.is_stop and not i.is_punct:
            filter.append(i.lemma_)
    return wv.get_mean_vector(filter)


In [11]:
df["vector"] = df["Text"].apply(preprocessing)

In [12]:
df.head()

,Text,label,vector
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,0,"[0.008657642, 0.019024342, -0.011917442, 0.032..."
1,U.S. conservative leader optimistic of common ...,1,"[0.010864096, 0.007960429, 0.0011915653, 0.014..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",1,"[0.018134918, 0.0062743523, -0.005872244, 0.03..."
3,Court Forces Ohio To Allow Millions Of Illega...,0,"[0.01255197, 0.012613623, 5.9780963e-05, 0.021..."
4,Democrats say Trump agrees to work on immigrat...,1,"[-0.0019059887, 0.011889367, 0.0035395357, 0.0..."


In [13]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(df.vector.values,df.label,random_state= 5,test_size= .2)

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train_stacked = np.stack(x_train)
x_test_stacked = np.stack(x_test)

In [15]:
x_train_standardise = scaler.fit_transform(x_train_stacked)
x_test_standardise = scaler.transform(x_test_stacked)

In [16]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(metric= "euclidean", n_neighbors= 5)
model.fit(x_train_standardise, y_train)

KNeighborsClassifier(metric='euclidean')

In [17]:
y_pred = model.predict(x_test_standardise)

In [18]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96      1001
           1       0.95      0.98      0.96       979

    accuracy                           0.96      1980
   macro avg       0.96      0.96      0.96      1980
weighted avg       0.96      0.96      0.96      1980

